# 1.  Imports and Data

In [1]:
from sklearn.decomposition import PCA
import matplotlib as mpl 
import numpy as np
import os
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler



from codes.utils import stratified_train_test_group_kfold
from codes.utils import pred2class
from codes.utils import model_test_regression
from codes.regression_codes import gpr_gridsearch



with open("Data/MoS2_Analysis_Data_trained2", "rb") as fp:   # Unpickling
    MoS2_Proj1_Class_Data = pickle.load(fp)

#features = MoS2_ImageNet_100_data[0]
df = pd.DataFrame(MoS2_Proj1_Class_Data)
print(df.keys())
T_target = np.array(list(df['T']))
features = np.array(list(df['ImageNet']))
sampleId = np.array(list(df['sampleId']))
train_val_sampleId, test_sampleId, train_val_Y, test_Y = train_test_split(sampleId, T_target, test_size=0.1, stratify = T_target, random_state=40)

0.10.1


# 2. Data Splitting and Scaling

In [2]:


X = features
Y = np.array(T_target)
groups = np.array(sampleId)




train_val_groups, train_val_X, train_val_Y, test_X, test_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=0)
train_val_Y = train_val_Y.flatten()
#val_Y = val_Y.flatten()
test_Y = test_Y.flatten()

print(train_val_X.shape)
print(train_val_Y.shape)
print(test_X.shape)
print(test_Y.shape)


oversample = RandomOverSampler(sampling_strategy='not majority')

scaler = StandardScaler().fit(train_val_X)
train_val_X = scaler.transform(train_val_X)
test_X = scaler.transform(test_X)

Index(['sampleId', 'sampleLabel', 'image', 'T', 'ImageNet', 'MicroNet'], dtype='object')


In [6]:
! mkdir -p 'Model/Reg/TrainedImageNet/Up'

# 3. Runing Cross-Validation

In [6]:
N_neighbors = [i for i in range(3, 15)]
Weights = ['uniform', 'distance']
Algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
P = [i for i in range(1, 6)]


def cross_10_folds_knr(train_val_X, train_val_Y):
    
    best_train = []
    best_val = []
    rmse_val = []
    best_variables = []
    for fold in range(10):
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(train_val_X, train_val_Y, train_val_groups, n_splits=10, test_fold=fold)
        model_path = f'regression/ImageNet/aug3/KNR_{fold}_model.sav'
        
        train_X, train_Y = oversample.fit_resample(train_X, train_Y)
        variables_best, performance_best = knr_gridsearch(train_X, train_Y, val_X, val_Y, N_neighbors, Weights, Algorithm, P, fold, model_path)
        best_train.append(performance_best['accuracy_train'])
        best_val.append(performance_best['accuracy_val'])
        rmse_val.append(performance_best['rmse_val'])
        best_variables.append(variables_best)
        print(f'fold: {fold} done!')
    return best_train, best_val, rmse_val, best_variables


best_train, best_val, rmse_val, best_variables = cross_10_folds_knr(train_val_X, train_val_Y)

with open("Model/Reg/TrainedImageNet/Up/KNN_RegModels_T_best_variables", "wb") as fb:
    pickle.dump(best_variables, fb)


print(best_train)
print(best_val)
print(rmse_val)
print(f'train_acc_mean: {np.mean(best_train) :.3f}, std: {np.std(best_train) :.3f}, val_acc_mean: {np.mean(best_val):.3f}, std: {np.std(best_val):.3f}, val_rmse: {np.mean(rmse_val):.3f}, std: {np.std(rmse_val):.3f}')

<class 'generator'>
best_train_acc: 0.863, best_val_acc: 0.708
best_train_acc: 1.000, best_val_acc: 0.750
best_train_acc: 1.000, best_val_acc: 0.792
best_train_acc: 0.759, best_val_acc: 0.833
fold: 0 done!
<class 'generator'>
best_train_acc: 0.843, best_val_acc: 0.792
best_train_acc: 0.852, best_val_acc: 0.875
best_train_acc: 1.000, best_val_acc: 0.917
fold: 1 done!
<class 'generator'>
best_train_acc: 0.846, best_val_acc: 0.500
best_train_acc: 1.000, best_val_acc: 0.542
best_train_acc: 1.000, best_val_acc: 0.583
best_train_acc: 0.808, best_val_acc: 0.625
best_train_acc: 0.783, best_val_acc: 0.708
fold: 2 done!
<class 'generator'>
best_train_acc: 0.848, best_val_acc: 0.583
best_train_acc: 0.860, best_val_acc: 0.667
best_train_acc: 0.724, best_val_acc: 0.708
best_train_acc: 0.654, best_val_acc: 0.750
fold: 3 done!
<class 'generator'>
best_train_acc: 0.860, best_val_acc: 0.708
best_train_acc: 0.790, best_val_acc: 0.750
fold: 4 done!
<class 'generator'>
best_train_acc: 0.863, best_val_acc:

# 4. Model Testing

In [5]:
best_fold = 1


model_path = f'regression/ImageNet/aug3/KNR_{best_fold}_model.sav'

# Fitting different train in the same hyperparameters
best_test, root_mean_squared_error, confusion_matrix_test = model_test_regression(train_val_X, train_val_Y, train_val_groups,test_X, test_Y, best_fold, model_path)

print('best_test: ', best_test)
print('rmse: ', root_mean_squared_error)
print(f'test_acc_mean: {np.mean(best_test) :.3f}, std: {np.std(best_test) :.3f}, rmse_mean: {np.mean(root_mean_squared_error)}, rmse_std: {np.std(root_mean_squared_error)}')

print("......")

print('5th confusion matrix: ', [list(item) for item in confusion_matrix_test[4]])

<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
best_test:  [0.5555555555555556, 0.6296296296296297, 0.6296296296296297, 0.48148148148148145, 0.6666666666666666, 0.5185185185185185, 0.5555555555555556, 0.5925925925925926, 0.5185185185185185, 0.6296296296296297]
rmse:  [34.907261631897875, 28.765545446741452, 28.075308062699506, 32.38125016858805, 28.627514539077513, 31.23663664962887, 36.100447385390076, 33.739931557842816, 32.502785397122025, 33.99240539302281]
test_acc_mean: 0.578, std: 0.058, rmse_mean: 32.032908623201095, rmse_std: 2.6546464915918464
......
5th confusion matrix:  [[3, 0, 0], [3, 10, 0], [0, 6, 5]]


In [6]:
cm_list = []
for cm in confusion_matrix_test:
    cm_list.append([list(item) for item in cm])
#print(cm_list)

print(len(cm_list))
print('avg cm: ', [list(item) for item in np.mean(cm_list, axis=0)])


10
avg cm:  [[2.0, 1.0, 0.0], [4.2, 7.5, 1.3], [0.3, 4.6, 6.1]]


In [3]:
best_fold = 1
#PATH = os.path.join('Model', model_path)
PATH = f'Model/Reg/TrainedImageNet/Up/KNN_{best_fold}_model.sav'
loaded_model = pickle.load(open(PATH, 'rb'))
loaded_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 6,
 'p': 4,
 'weights': 'distance'}